## Notes : please use python version > 3.10 or 3.11 to run all the codes below 

# Installations

# Import Library

In [1]:
# import library 
import os
import pandas as pd
import numpy as np

#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform
from google.auth.exceptions import DefaultCredentialsError

# ML algorithm
import xgboost as xgb
import category_encoders as ce
import xgboost as xgb
from sklearn.compose import ColumnTransformer
from sklearn.compose import TransformedTargetRegressor

## sklearn module
import pickle
import warnings
warnings.filterwarnings('ignore')

### Authenticate to GCP using Services Account 

- Depending on your Jupyter environment, you may have to manually 

- Call "sa-development.json" as teh auth to GCP 

- Adjust your path where the all code stored in you local 

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/home/elisamelinda85/elisa-008/sa-development.json"

## Replace the code below with yours configuration 
- Configuratin naming convention has shared on the following link [configuration name](https://docs.google.com/spreadsheets/d/1U7bbXp9Y6uLGZXThfsqDneo1U-E3J8nwCcxIdpjgZmI/edit#gid=1834500505)

- Example configuration

    ```
    project_id = 'dti-ds'
    dataset_id = 'jaya_dataset_000'
    table_id = 'german_dataset'
    region = 'us-central1'
    bucket_name = 'jaya_gcs_000'
    blob_name = 'data/german_dataset.csv'
    
    ```


In [3]:
project_id = 'dti-ds'
dataset_id = 'elisa_dataset_008'
table_id = 'daegu_apartment_data_test'
region = 'us-central1'
bucket_name = 'elisa_gcs_008'

In [4]:
model_name = 'DaeguApartment_Final_XGB.sav'
local_model_dir = 'models'
# local_model_path = './DaeguApartment_Final_XGB.sav'
local_model_path = os.path.join(local_model_dir, model_name)

# Ensure the local directory exists
os.makedirs(local_model_dir, exist_ok=True)

try:
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name)
    blob_model = bucket.blob(f"model/{model_name}")
    blob_model.download_to_filename(local_model_path)

    print("Retrieve model succeeded")
except DefaultCredentialsError as e:
    print(f"Credentials error: {e}")
except FileNotFoundError as e:
    print(f"File not found: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

Retrieve model succeeded


## Model Predictions

In [5]:
model_name = "DaeguApartment_Final_XGB.sav"
model_path = f"models/{model_name}"

loaded_model = pickle.load(open(model_path, 'rb'))

# Sample data for prediction
test_data = pd.DataFrame(
    {
    'HallwayType' : ['terraced','mixed'],
    'TimeToSubway' : ['0~5min','10min~15min'],
    'SubwayStation' : ['Kyungbuk_uni_hospital','Myung-duk'],
    'OtherFacilitiesNearby' : [2,0],
    'PublicOfficeNearby' : [2,0],
    'UniversityNearby' : [2,0],
    'ParkinglotBasement' : [100,0],
    'YearBuilt' : [2006,1976],
    'AptFacilities' : [2,0],
    'Size_sqf' : [1000,206]}
)

predictions = loaded_model.predict(test_data)

predictions

array([234624.23,  63837.22], dtype=float32)

In [6]:
# load data from BQ
client = bigquery.Client(project=project_id)

# query 
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")
test_df = query_job.result().to_dataframe()

test_df

,HallwayType,TimeToSubway,SubwayStation,OtherFacilitiesNearby,PublicOfficeNearby,UniversityNearby,ParkinglotBasement,YearBuilt,AptFacilities,Size_sqf
0,terraced,0~5min,Banwoldang,0,0,0,203,2014,10,914
1,terraced,0~5min,Banwoldang,0,0,0,203,2014,10,1252
2,terraced,0~5min,Banwoldang,0,0,0,203,2014,10,1256
3,terraced,0~5min,Banwoldang,0,0,0,203,2014,10,914
4,terraced,0~5min,Banwoldang,0,0,0,203,2014,10,914
...,...,...,...,...,...,...,...,...,...,...
777,terraced,0~5min,Myung-duk,5,7,5,1174,2014,9,910
778,terraced,0~5min,Myung-duk,5,7,5,1174,2014,9,914
779,terraced,0~5min,Myung-duk,5,7,5,1174,2014,9,910
780,terraced,0~5min,Myung-duk,5,7,5,1174,1980,9,1060


In [7]:
# rename column name for better understanding
test_df.rename(columns={'Parkinglot_Basement_':'Parkinglot(Basement)','Size_sqf_':'Size(sqf)'},inplace=True)

In [8]:
bulk_predict_df = test_df

In [9]:
y_pred = loaded_model.predict(bulk_predict_df)
y_pred_df = pd.DataFrame(y_pred, columns=['PricePrediction'])

result_df = pd.concat([test_df.reset_index(drop=True), y_pred_df.reset_index(drop=True)], axis=1)
result_df

,HallwayType,TimeToSubway,SubwayStation,OtherFacilitiesNearby,PublicOfficeNearby,UniversityNearby,ParkinglotBasement,YearBuilt,AptFacilities,Size_sqf,PricePrediction
0,terraced,0~5min,Banwoldang,0,0,0,203,2014,10,914,358602.687500
1,terraced,0~5min,Banwoldang,0,0,0,203,2014,10,1252,447873.750000
2,terraced,0~5min,Banwoldang,0,0,0,203,2014,10,1256,447873.750000
3,terraced,0~5min,Banwoldang,0,0,0,203,2014,10,914,358602.687500
4,terraced,0~5min,Banwoldang,0,0,0,203,2014,10,914,358602.687500
...,...,...,...,...,...,...,...,...,...,...,...
777,terraced,0~5min,Myung-duk,5,7,5,1174,2014,9,910,346343.281250
778,terraced,0~5min,Myung-duk,5,7,5,1174,2014,9,914,351845.750000
779,terraced,0~5min,Myung-duk,5,7,5,1174,2014,9,910,346343.281250
780,terraced,0~5min,Myung-duk,5,7,5,1174,1980,9,1060,100694.335938


In [10]:
from google.cloud.exceptions import NotFound

dest_table_id = 'daegu_apartment_price_prediction'
table_full_id = f'{project_id}.{dataset_id}.{dest_table_id}'

schema = [
    bigquery.SchemaField('HallwayType', 'STRING'),
    bigquery.SchemaField('TimeToSubway', 'STRING'),
    bigquery.SchemaField('SubwayStation', 'STRING'),
    bigquery.SchemaField('OtherFacilitiesNearby', 'INTEGER'),
    bigquery.SchemaField('PublicOfficeNearby', 'INTEGER'),
    bigquery.SchemaField('UniversityNearby', 'INTEGER'),
    bigquery.SchemaField('ParkinglotBasement', 'INTEGER'),
    bigquery.SchemaField('YearBuilt', 'INTEGER'),
    bigquery.SchemaField('AptFacilities', 'INTEGER'),
    bigquery.SchemaField('Size_sqf', 'INTEGER'),
]

table_ref = client.dataset(dataset_id).table(dest_table_id)

# Check if the table exists
try:
    client.get_table(table_ref)
    print(f'Table {table_full_id} already exists.')
except NotFound:
    # Create the table if it does not exist
    table = bigquery.Table(table_ref, schema=schema)
    table = client.create_table(table)
    print(f'Table {table_full_id} created.')

job_config = bigquery.LoadJobConfig(
    schema=schema,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE
)

load_job = client.load_table_from_dataframe(result_df, table_ref, job_config=job_config)
load_job.result()

print(f'Loaded {load_job.output_rows} rows into {table_full_id}')

Table dti-ds.elisa_dataset_008.daegu_apartment_price_prediction already exists.
Loaded 782 rows into dti-ds.elisa_dataset_008.daegu_apartment_price_prediction
